In [1]:
import scipy.io as io
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import bernoulli
from scipy.stats import multinomial
from collections import OrderedDict

In [2]:
np.random.seed(1)
mat = io.loadmat('tempNs4.mat')
model = {}
model['H'] = mat['H']
model['Wopt'] = mat['Wopt']
model['Fopt'] = mat['Fopt']
model['Ns'] = mat['Ns'][0,0] ## need to change nb of stream in channel realization, this case 1
# model['Ns'] = 2
model['Nt'] = mat['Nt'][0,0]
model['Nr'] = mat['Nr'][0,0]
model['Nc'] = mat['Nc'][0,0]
model['Nray'] = mat['Nray'][0,0]
model['realization'] = mat['realization'][0,0]
model['At'] = mat['At']
model['Ar'] = mat['Ar']
model['Nrf'] = 4
archi = 'FC'

In [3]:
np.diagflat([1,2,3,4])

array([[1, 0, 0, 0],
       [0, 2, 0, 0],
       [0, 0, 3, 0],
       [0, 0, 0, 4]])

In [4]:
V_hat = np.zeros((model['Nt'],model['Ns'],model['realization']),dtype=complex)
U_hat = np.zeros((model['Nr'],model['Ns'],model['realization']),dtype=complex)
sigma_hat = np.zeros((model['Ns'],model['Ns'],model['realization']),dtype=complex)

In [5]:
for reali in range(model['realization']):
    if (np.linalg.matrix_rank(model['H'][:,:,reali])>=model['Ns']):
#         count = count + 1
        U, s, V = np.linalg.svd(model['H'][:,:,reali], full_matrices=True)
        V = V.conj().T
        V_hat[:,:,reali] = V[0:model['Nt'],0:model['Ns']]
        U_hat[:,:,reali] = U[0:model['Nr'],0:model['Ns']]
        sigma_hat[:,:,reali] = np.diagflat(s[:model['Ns']])

In [12]:
alpha = 0.0001
n_bit = 2
q = 2*np.pi/2**n_bit

In [13]:
F_rf_angle = np.zeros((model['Nt'],model['Nrf']))
W_rf_angle = np.zeros((model['Nr'],model['Nrf']))
F_rf = np.exp(1j*F_rf_angle)
W_rf = np.exp(1j*W_rf_angle)

In [14]:
def opt_frf(a,b,c,i,q=2*np.pi/2**2):
    b_cut = np.delete(b, i, axis=1)
    b_i = b[:,[i]]
    c_cut = np.delete(c, i, axis=0)

    big = a@b_cut@c_cut
    small = a@b_i

    ang = np.angle(big) - np.angle(small)
    ang = q * np.round(ang/q)
#     print(ang)
    c_new = np.exp(1j*ang)
    return c_new

In [15]:
def opt_wrf(a,b,c,i,q=2*np.pi/2**2):
    b_cut = np.delete(b, i, axis=0)
    b_i = b[[i],:]
    _a = a.conj().T
    _a_cut = np.delete(_a, i, axis=1)
    big = _a_cut@b_cut@c
    small = b_i@c

    ang = np.angle(big) - np.angle(small)
    ang = q * np.round(ang/q)
    _a[0][i] = np.exp(1j*ang)
    return _a



In [17]:
def data_rate(Ns,SNR,W,H,F):
    """
    Compute the data rate of the beamforming system
    
    Arguments:
    Ns -- number of stream
    SNR -- Signal to noise ratio
    W -- combiner
    H -- channel
    F -- precoder
    
    Returns:
    R -- data rate
    """
    R = np.log2(np.linalg.det(np.eye(Ns)+SNR/Ns*np.linalg.pinv(W)@H@F@F.conj().T@H.conj().T@W))
    return R.real

In [20]:
# SNR = 10**(10/10)
# data_rate(model['Ns'],SNR,W_rf/np.sqrt(model['Nr']),H0,F_rf/np.sqrt(model['Nt']))

In [21]:
# F_rf_angle = np.zeros((model['Nt'],model['Nrf']))
# W_rf_angle = np.zeros((model['Nr'],model['Nrf']))
# F_rf = np.exp(1j*F_rf_angle)
# W_rf = np.exp(1j*W_rf_angle)
# SNR = 10**(10/10)

# for t in range(5):
#     for l in range(model['Ns']):
# #         print('-----------')
#         Frf_l = np.delete(F_rf, l, axis=1)
#         Wrf_l = np.delete(W_rf, l, axis=1)
#         fl = F_rf[:,[l]]
#         wl = W_rf[:,[l]]
#         Ql = U_hat0@np.linalg.inv(alpha*np.eye(model['Ns']) + sigma_hat0@V_hat0.conj().T@Frf_l@Wrf_l.conj().T@U_hat0)@sigma_hat0@V_hat0.conj().T

#         for k in range(5):
#             for i in range(model['Nt']):
#                 fl[i] = opt_frf(wl.conj().T,Ql,fl,i)
#             for j in range(model['Nr']):
#                 wl = opt_wrf(wl,Ql,fl,j).conj().T
#         F_rf[:,[l]] = fl 
#         W_rf[:,[l]] = wl


In [22]:
def coordinate_descent(index,Nt=64,Nr=16,Ns=4,Nrf=4):
    F_rf_angle = np.zeros((Nt,Nrf))
    W_rf_angle = np.zeros((Nr,Nrf))
    F_rf = np.exp(1j*F_rf_angle)
    W_rf = np.exp(1j*W_rf_angle)
    
    U_hat0 = U_hat[:,:,index]
    V_hat0 = V_hat[:,:,index]
    sigma_hat0 = sigma_hat[:,:,index]
    
    for t in range(5):
        for l in range(Ns):
    #         print('-----------')
            Frf_l = np.delete(F_rf, l, axis=1)
            Wrf_l = np.delete(W_rf, l, axis=1)
            fl = F_rf[:,[l]]
            wl = W_rf[:,[l]]
            Ql = U_hat0@np.linalg.inv(alpha*np.eye(Ns) + sigma_hat0@V_hat0.conj().T@Frf_l@Wrf_l.conj().T@U_hat0)@sigma_hat0@V_hat0.conj().T

            for k in range(5):
                for i in range(model['Nt']):
                    fl[i] = opt_frf(wl.conj().T,Ql,fl,i)
                for j in range(model['Nr']):
                    wl = opt_wrf(wl,Ql,fl,j).conj().T
            F_rf[:,[l]] = fl 
            W_rf[:,[l]] = wl
    
    F_rf = F_rf/np.sqrt(Nt)
    W_rf = W_rf/np.sqrt(Nr)

    H_eff = W_rf.conj().T@model['H'][:,:,index]@F_rf
    U, s, V = np.linalg.svd(H_eff, full_matrices=True)
    V = V.conj().T
    F_bb = V
    W_bb = U
    
    F_bb = np.sqrt(Ns)*F_bb/np.linalg.norm(F_rf@F_bb,'fro')
    W_bb = np.sqrt(Ns)*W_bb/np.linalg.norm(W_rf@W_bb,'fro')
    output = {}
    
    output['F_rf'] = F_rf
    output['W_rf'] = W_rf
    output['F_bb'] = F_bb
    output['W_bb'] = W_bb
    
    return output

In [23]:
# index = 0
# test = coordinate_descent(index)

In [24]:
# SNR_dB = np.arange(-30,15,5)
# SNR_dB

In [25]:
# SNR_dB.shape[0]

In [26]:
# sum = np.zeros(SNR_dB.shape)
# sum

In [27]:
SNR_dB = np.arange(-30,15,5)
sum = np.zeros(SNR_dB.shape)
for index in range(20,25):
    test = coordinate_descent(index)
    for i in range(SNR_dB.shape[0]):
        rate = data_rate(model['Ns'],10**(SNR_dB[i]/10),test['W_rf']@test['W_bb'],model['H'][:,:,index],test['F_rf']@test['F_bb'])
        sum[i] += rate
    print(sum)

[ 0.17199009  0.5232222   1.48880305  3.70347709  7.62163187 12.9866658
 19.14509433 25.62445131 32.21505465]
[ 0.38802035  1.17765029  3.32980371  8.17118294 16.4796649  27.52574174
 39.98335095 52.99269224 66.19062058]
[ 0.5299351   1.61164686  4.58092673 11.36117797 23.26388246 39.44783525
 57.95868154 77.40879664 97.1844899 ]
[  0.67277979   2.04902688   5.8449215   14.59137649  30.12877375
  51.4796915   76.05664388 101.95210094 128.30705859]
[  0.83049962   2.53178029   7.23778554  18.12942559  37.54580506
  64.25433852  94.990714   127.36651101 160.31246871]


In [28]:
# test['W_rf'].conj().T@model['H'][:,:,10]@test['F_rf']

In [29]:
# Normalize condition
x = np.linalg.norm(test['F_rf']@test['F_bb'],'fro')
print("Ns", model['Ns'] , "   Frobenius norm FRF*FBB=", x**2)

Ns 4    Frobenius norm FRF*FBB= 4.0


In [30]:
sum/5

array([ 0.16609992,  0.50635606,  1.44755711,  3.62588512,  7.50916101,
       12.8508677 , 18.9981428 , 25.4733022 , 32.06249374])